# Generator step 1 - Initial data finding

[x] Function calling

[ ] Some way to manage facts - add results from tools calls, convert to XML, ...

[ ] Test if we have unfilled facts and or we got some new info in the last update (in previous task)

[ ] Add source info for each stored fact

[ ] Iteratively fill the fact sheet - convert from chain to graph


The compiler needs to add some internal facts - in this case customer ID.

In [17]:
ASSET_REPORT = {
    'facts':{
        'today': 'Current date',
        'portfolio_today': 'List of holdings on specific day',
        'portfolio_last': 'List of holdings one month before report day',
        'client_id': 'Numerical ID for client'
    }
}

In [2]:
from langchain_core.tools import tool

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai.model_garden import ChatAnthropicVertex

from yava.models import ToolDef, Tools

In [3]:
from datetime import date

@tool
def current_date() -> date:
    """
    Return today`s date
    """
    return date.today()

In [4]:
@tool
def get_customer_id(name: str) -> int:
    """
    Get client id from the name
    """
    return 1

In [5]:
@tool
def holdings(customer: str, day: date):
    """
    Get list of holdings of client on specific day
    """
    if day.date >= 8:
        return [{'symbol': 'GOOGL',  'quantity': 100, 'cost_basis': 110}]       
    else:
        return [{'symbol': 'GOOGL',  'quantity': 90, 'cost_basis': 105}]       

In [6]:
tools = Tools(tools = {
    current_date.name : ToolDef(tool = current_date, output = 'today'),
    get_customer_id.name: ToolDef(tool = get_customer_id, output = 'customer_id'),
    holdings.name : ToolDef(tool = holdings, output = 'portfolio_today')
})

In [7]:
gemini = ChatVertexAI(model_name='gemini-1.5-pro-001')
sonnet = ChatAnthropicVertex(model='claude-3-5-sonnet@20240620', location='us-east5')

In [8]:
facts = '''<FACT><NAME>today</NAME><DESCRIPTION>Today`s date</DESCRIPTION></FACT>
<FACT><NAME>client_id</NAME><DESCRIPTION>Cliend ID</DESCRIPTION><FACT>
<FACT><NAME>portfolio_today</NAME><DESCRIPTION>List of holdings on specific day</DESCRIPTION></FACT>
'''

In [9]:
prompt = ChatPromptTemplate(
    messages=[
        ('system', 'You are an assistant collecting data for an report. Gather facts to fill in facts structure. Request only unknown facts'),
        ('user', 'Create asset report for {customer}\nKnown facts: {known}. All facts: {facts}')
    ]
)

In [10]:
chain = prompt | gemini.bind_tools(tools.get_tools())

In [15]:
response = chain.invoke(input={'customer': 'Boris', 'facts' : facts, 'known': '<today>08/12/2024</today><customer_id>1</customer_id>'})

In [16]:
response.tool_calls

[{'name': 'holdings',
  'args': {'day': '08/12/2024', 'customer': '1'},
  'id': 'e7aca2f2-61dd-4766-a2a2-65999833f758',
  'type': 'tool_call'}]